In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Time
import time

# Random and os for reproducibility
import random
import os

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
import tensorflow as tf

# Processing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

# Pipeline
from sklearn.pipeline import Pipeline

# Data imputation
from sklearn.impute import SimpleImputer

# Making keras compatible with scikit learn api
# https://scikit-learn.org/stable/developers/develop.html
from sklearn.base import BaseEstimator, RegressorMixin

# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

# Converting Times Series Data to 3D format
from src.utils.split_sequences import split_sequences

2024-04-07 16:20:11.484827: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-07 16:20:11.489953: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-07 16:20:11.561207: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-07 16:20:11.563100: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-04-07 16:20:12.686597: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


<IPython.core.display.Javascript object>

# Functions and definitions

## Helper functions for blocked time series cross validation

### Convert train/test data to 3D format

In [3]:
def generate_sequences_helper(
    dataset, cement_types, dates=None, timesteps=None, split_by_cement_type=False
):
    index_train = dataset["y_train"].index
    index_test = dataset["y_test"].index

    dataset["y_train"] = dataset["y_train"].reset_index(drop=True)
    dataset["y_test"] = dataset["y_test"].reset_index(drop=True)

    if dates is not None:
        dataset["dates_train"] = dates[index_train].reset_index(drop=True)
        dataset["dates_test"] = dates[index_test].reset_index(drop=True)

    dataset["cement_types_train"] = cement_types.loc[index_train].reset_index(drop=True)
    dataset["cement_types_test"] = cement_types.loc[index_test].reset_index(drop=True)

    dataset = generate_sequences(dataset, timesteps, split_by_cement_type)

    return dataset

<IPython.core.display.Javascript object>

In [4]:
def generate_sequences(dataset, timesteps, split_by_cement_type=False):
    if split_by_cement_type:
        dataset["x_train"], dataset["y_train"] = split_sequences_per_cement_type(
            pd.concat(
                [
                    dataset["dates_train"],
                    pd.DataFrame(dataset["x_train"], columns=x.columns),
                    dataset["cement_types_train"],
                    dataset["y_train"],
                ],
                axis=1,
            ),
            timesteps,
        )

        dataset["x_test"], dataset["y_test"] = split_sequences_per_cement_type(
            pd.concat(
                [
                    dataset["dates_test"],
                    pd.DataFrame(dataset["x_test"], columns=x.columns),
                    dataset["cement_types_test"],
                    dataset["y_test"],
                ],
                axis=1,
            ),
            timesteps,
        )
    else:
        dataset["x_train"], dataset["y_train"] = split_sequences(
            pd.concat(
                [
                    pd.DataFrame(dataset["x_train"], columns=x.columns),
                    dataset["y_train"],
                ],
                axis=1,
            ).values,
            timesteps,
        )

        dataset["x_test"], dataset["y_test"] = split_sequences(
            pd.concat(
                [
                    pd.DataFrame(dataset["x_test"], columns=x.columns),
                    dataset["y_test"],
                ],
                axis=1,
            ).values,
            timesteps,
        )
    return dataset

<IPython.core.display.Javascript object>

### Data preprocessing

In [5]:
def impute_data(dataset, imputer=None, imputer_params=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply imputation to the data
    if imputer is not None:
        imputer = imputer() if imputer_params is None else imputer(**imputer_params)
        x_train = imputer.fit_transform(x_train)
        x_test = imputer.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [6]:
def transform_data(dataset, transformer=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply data normalization/standardization to the data
    if transformer is not None:
        scaler = transformer()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [7]:
def preprocess_data(dataset, transformer=None, imputer=None, imputer_params=None):
    dataset = impute_data(dataset, imputer, imputer_params)
    dataset = transform_data(dataset, transformer)
    return dataset

<IPython.core.display.Javascript object>

### Train and evaluate the model

In [8]:
def train_and_evaluate_model(Estimator, dataset, estimator_params=None):
    """
    Purpose: Helper function to be used in conjunction with
    blocked time_series cross validation function
    """
    x_train = dataset["x_train"]
    y_train = dataset["y_train"]
    x_test = dataset["x_test"]
    y_test = dataset["y_test"]

    # Instantiate the model
    model = Estimator() if estimator_params is None else Estimator(**estimator_params)

    # Fitting the model
    model.fit(x_train, y_train)

    # Making predictions on train/test sets
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Return regression metrics
    return score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

<IPython.core.display.Javascript object>

### Custom Cross Validate

In [9]:
def custom_cross_validate(
    Estimator,
    Imputer,
    Transform,
    x,
    y,
    cv,
    timesteps,
    dates=None,
    cement_types=None,
    estimator_params=None,
    imputer_params=None,
    split_by_cement_type=True,
):
    results = []
    scores = []

    for train_index, test_index in cv.split(x):
        dataset = {
            "dates_train": dates[train_index].reset_index(drop=True),
            "cement_types_train": cement_types.loc[train_index].reset_index(drop=True),
            "x_train": x.loc[train_index].reset_index(drop=True),
            "y_train": y[train_index].reset_index(drop=True),
            "dates_test": dates[test_index].reset_index(drop=True),
            "cement_types_test": cement_types.loc[test_index].reset_index(drop=True),
            "x_test": x.loc[test_index].reset_index(drop=True),
            "y_test": y[test_index].reset_index(drop=True),
        }

        # Preprocess the dataset
        dataset = preprocess_data(dataset, Transform, Imputer, imputer_params)

        # generate sequences (3D format)
        dataset = generate_sequences(dataset, timesteps, split_by_cement_type)

        # Train and Evaluate the model
        score = train_and_evaluate_model(Estimator, dataset, estimator_params)
        scores.append(score)

    # After every iteration metrics results are appended together
    scores_final = {key: [] for key, _ in scores[0].items()}
    for scores_dict in scores:
        for key, value in scores_dict.items():
            scores_final[key] += [value]
    results.append(scores_final)
    return results

<IPython.core.display.Javascript object>

## Model Definition

In [10]:
class BidirectionalGRU(BaseEstimator, RegressorMixin):
    def __init__(self):
        self.model = self.get_model()
        self.batch_size = 16
        self.epochs = 300
        self.verbose = 0

    def fit(self, X=None, y=None):
        self.model.fit(
            X, y, batch_size=self.batch_size, epochs=self.epochs, verbose=self.verbose
        )

    def predict(self, X=None):
        return self.model.predict(X, verbose=self.verbose)

    def get_model(self):
        model = tf.keras.Sequential()
        model.add(
            tf.keras.layers.Bidirectional(
                tf.keras.layers.GRU(units=16, activation="relu")
            )
        ),
        model.add(tf.keras.layers.Dropout(rate=0.10))
        model.add(tf.keras.layers.Dense(units=1))
        model.compile(
            optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
            loss="mse",
            metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
        )
        return model

<IPython.core.display.Javascript object>

In [11]:
def pad_time_series(dataframe, timesteps):
    """
    Pad timeseries with zeros
    """
    df_tmp = pd.DataFrame(
        dict(
            zip(
                dataframe.columns,
                [[0 for _ in range(timesteps - 1)] for _ in range(dataframe.shape[1])],
            )
        )
    )
    df_tmp[DATE] = dataframe[DATE].iloc[0]
    return pd.concat([df_tmp, dataframe], axis=0).reset_index(drop=True)

<IPython.core.display.Javascript object>

In [12]:
def split_sequences_per_cement_type(dataframe, timesteps, pad=False):
    """
    Create sequences per cement time
    to avoid having parts of the sequence
    of different types of cement.
    """
    if timesteps == 1:
        return split_sequences(
            dataframe.drop([DATE] + CEMENT_TYPES, axis=1).values, timesteps
        )

    dates = dataframe[DATE][timesteps - 1 :]
    data = []
    dataframes = []

    for cement_type in CEMENT_TYPES:
        data.append(dataframe[dataframe[cement_type] == 1])
    data.append(dataframe[(dataframe[CEMENT_TYPES] == 0).all(axis=1)])

    for df in data:
        if pad:
            dates = df[DATE].reset_index(drop=True)
            df = pad_time_series(df, timesteps).reset_index(drop=True)
        else:
            dates = df[DATE][timesteps - 1 :].reset_index(drop=True)
        x, y = split_sequences(df.drop([DATE] + CEMENT_TYPES, axis=1).values, timesteps)
        x = pd.DataFrame({"Sequences": [sample.tolist() for sample in x]})
        y = pd.DataFrame({"Target": y})
        dataframes.append(pd.concat([dates, x, y], axis=1))

    data = pd.concat(dataframes, axis=0)
    data[DATE] = pd.to_datetime(data[DATE])
    data = data.sort_values(by=DATE).reset_index(drop=True)
    x = data["Sequences"]
    y = data["Target"].values
    x = np.array(x.tolist())

    return x, y

<IPython.core.display.Javascript object>

# Settings for Reproducibility

In [13]:
def set_seeds():
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
    os.environ["PYTHONHASHSEED"] = str(SEED)
    tf.random.set_seed(SEED)
    np.random.seed(SEED)
    random.seed(SEED)


# https://stackoverflow.com/questions/36288235/how-to-get-stable-results-with-tensorflow-setting-random-seed

<IPython.core.display.Javascript object>

In [14]:
def set_global_determinism():
    set_seeds(seed=SEED)

    os.environ["TF_DETERMINISTIC_OPS"] = "1"
    os.environ["TF_CUDNN_DETERMINISTIC"] = "1"

    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

<IPython.core.display.Javascript object>

In [15]:
index_to_save = 8

<IPython.core.display.Javascript object>

In [16]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}
DATE = "Date"
CEMENT_TYPES = [
    "Cement_Type_CP III32",
    "Cement_Type_CP VARI",
    "Cement_Type_Fibrocimento",
]

<IPython.core.display.Javascript object>

## Defining a dataframe structure to save the results

In [17]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "203",
    "Plant": "AV",
    "Features": "Chemical + CS1 + CS3 + CS7",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "BidirectionalGRU",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [18]:
df = pd.read_csv("../../../../../../data/processed/203/av.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we keep only chemical and mineralogical features yielded by the same testing method/procedure

In [19]:
df_copy = df.copy()
df_copy = pd.get_dummies(data=df_copy, columns=["Cement_Type"], drop_first=True)

df_copy = df_copy.drop(
    [
        "Blaine",
        "#200",
        "#325",
        "Final setting time",
        "Initial setting time",
    ],
    axis=1,
)

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we use all available features

<h2>1. Dataset: df_no_cs</h2> <br>In this dataset the CS1, CS3  and CS7 variables are not considered. Only Chemical and mineralogical features measured by the same method. For this particular dataset, all chemical features, with the exception of LOI were measured by XRF and XRD methods.

In [20]:
df_copy[CEMENT_TYPES] = df_copy[CEMENT_TYPES].astype(int)
dates = df["Date"].copy()
y = df_copy.pop("CS28")
x = df_copy
df_copy = pd.concat([x, y], axis=1)

<IPython.core.display.Javascript object>

In [21]:
TIMESTEPS_LIST = [1, 7, 14]

<IPython.core.display.Javascript object>

# 1. Bidirectional Gated Recurrent Unit - Bi-GRU

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Timesteps:</b> 1, 3, 5, 7, 10, 15, 20<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>10<br>
    1. 10 folds of 123 samples each
    2. 90% train (1111 samples each fold)
    3. 10% test (123 samples each fold)
<b>Total:</b> 100 models<br>

In [22]:
start = time.time()

repeats = 3
n_splits = 5
TIMESTEPS_LIST = [1, 7, 14]

print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
    x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
    y = df_copy["CS28"]
    scores = custom_cross_validate(
        BidirectionalGRU,
        SimpleImputer,
        StandardScaler,
        x,
        y,
        cv,
        timesteps=timesteps,
        dates=dates,
        cement_types=df_copy[CEMENT_TYPES],
        estimator_params=None,
        imputer_params={"strategy": "median"},
        split_by_cement_type=True,
    )
    scores = scores[0]
    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores, METRICS, METRICS_DICT)

    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Repeated KFold"
    results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Repeated Cross Validation:
Repeats: 3
n_splits: 5



2024-04-07 16:20:17.960154: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-04-07 16:20:17.960203: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: inspirada
2024-04-07 16:20:17.960209: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: inspirada
2024-04-07 16:20:17.960414: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 525.147.5
2024-04-07 16:20:17.960442: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 525.147.5
2024-04-07 16:20:17.960447: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 525.147.5


TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 1.719 (0.036)
MAE: 1.339 (0.034)
MAPE: 0.027 (0.001)
R2: 0.820 (0.010)


******
[TEST]
******
RMSE: 1.849 (0.115)
MAE: 1.434 (0.077)
MAPE: 0.029 (0.001)
R2: 0.790 (0.025)




TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.423 (0.130)
MAE: 1.137 (0.125)
MAPE: 0.023 (0.003)
R2: 0.878 (0.022)


******
[TEST]
******
RMSE: 1.956 (0.138)
MAE: 1.532 (0.128)
MAPE: 0.032 (0.003)
R2: 0.768 (0.035)




TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.206 (0.083)
MAE: 0.948 (0.070)
MAPE: 0.019 (0.001)
R2: 0.913 (0.014)


******
[TEST]
******
RMSE: 1.984 (0.171)
MAE: 1.520 (0.135)
MAPE: 0.031 (0.003)
R2: 0.744 (0.043)


Minutes Elapsed:  99.31462670564652


<IPython.core.display.Javascript object>

In [23]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features             Model Cross Validation Timesteps  \
                                                                             
0  Chemical + CS1 + CS3 + CS7  BidirectionalGRU   Repeated KFold         1   
1  Chemical + CS1 + CS3 + CS7  BidirectionalGRU   Repeated KFold         7   
2  Chemical + CS1 + CS3 + CS7  BidirectionalGRU   Repeated KFold        14   

  RMSE Test            MAE Test           MAPE Test             R2 Test  \
       mean       std      mean       std      mean       std      mean   
0  1.849068  0.114643  1.434239  0.077411  0.029153  0.001466  0.790342   
1  1.955524  0.137611  1.531781  0.127570  0.031518  0.002694  0.767533   
2  1.983682  0.171068  1.519980  0.134988  0.030562  0.002633  0.744106   

             
        std  
0  0.024612  
1  0.034939  
2  0.042596

<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 246 samples
    2. 50% train (123 samples each fold)
    3. 50% test (123 samples each fold)
<b>Total:</b> 5 models<br>

In [24]:
start = time.time()

repeats = 3
n_splits = 5
train_size = 0.8
TIMESTEPS_LIST = [1, 7, 14]

print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    for _ in range(repeats):
        x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
        y = df_copy["CS28"]

        cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)

        scores = custom_cross_validate(
            BidirectionalGRU,
            SimpleImputer,
            StandardScaler,
            x,
            y,
            cv,
            timesteps,
            dates=dates,
            cement_types=df_copy[CEMENT_TYPES],
            estimator_params=None,
            imputer_params={"strategy": "median"},
            split_by_cement_type=True,
        )
        scores = scores[0]
        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    scores = {key: np.array(val).flatten() for key, val in scores_final.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
    results_dict_copy[
        "Cross Validation Params"
    ] = '{"N_Splits": 5, "Repeats": 3, "train_size": 0.8}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Blocking Time Series Split:
Repeats: 3
n_splits: 5



TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 1.712 (0.374)
MAE: 1.339 (0.259)
MAPE: 0.027 (0.005)
R2: 0.055 (0.583)


******
[TEST]
******
RMSE: 5.061 (4.620)
MAE: 3.846 (3.340)
MAPE: 0.082 (0.075)
R2: -6.260 (13.822)




TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.373 (0.394)
MAE: 1.094 (0.305)
MAPE: 0.022 (0.006)
R2: 0.350 (0.478)


******
[TEST]
******
RMSE: 8.416 (8.409)
MAE: 6.744 (6.626)
MAPE: 0.144 (0.148)
R2: -19.694 (34.956)




TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.223 (0.211)
MAE: 0.990 (0.178)
MAPE: 0.020 (0.004)
R2: 0.472 (0.307)


******
[TEST]
******
RMSE: 7.766 (4.588)
MAE: 6.509 (3.581)
MAPE: 0.136 (0.076)
R2: -13.951 (17.362)


Minutes Elapsed:  24.08261617819468


<IPython.core.display.Javascript object>

In [25]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features             Model            Cross Validation  \
                                                                              
0  Chemical + CS1 + CS3 + CS7  BidirectionalGRU  Blocking Time Series Split   
1  Chemical + CS1 + CS3 + CS7  BidirectionalGRU  Blocking Time Series Split   
2  Chemical + CS1 + CS3 + CS7  BidirectionalGRU  Blocking Time Series Split   
3  Chemical + CS1 + CS3 + CS7  BidirectionalGRU              Repeated KFold   
4  Chemical + CS1 + CS3 + CS7  BidirectionalGRU              Repeated KFold   
5  Chemical + CS1 + CS3 + CS7  BidirectionalGRU              Repeated KFold   

  Timesteps RMSE Test            MAE Test           MAPE Test            \
                 mean       std      mean       std      mean       std   
0         1  5.060985  4.620092  3.845612  3.340128  0.081588  0.074701   
1         7  8.416255  8.409021  6.744304  6.625865  0.144470  0.147576   
2        14  7.765740  4.588193  6.509323  3.581415  0.136334  0.076461   
3         1  1.849068  0.114643  1.434239  0.077411  0.029153  0.001466   
4         7  1.955524  0.137611  1.531781  0.127570  0.031518  0.002694   
5        14  1.983682  0.171068  1.519980  0.134988  0.030562  0.002633   

     R2 Test             
        mean        std  
0  -6.260210  13.822215  
1 -19.693881  34.956397  
2 -13.950874  17.362201  
3   0.790342   0.024612  
4   0.767533   0.034939  
5   0.744106   0.042596

<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 114, 226, 338, 450, 562, 675, 787, 899, 1011, 1123 samples each fold
    2. Test: 112 samples each fold
<b>Total:</b> 10 models<br>

In [26]:
set_seeds()
start = time.time()
gap = 0
n_splits = 5
repeats = 3
TIMESTEPS_LIST = [1, 7, 14]

print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    for _ in range(repeats):
        x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
        y = df_copy["CS28"]

        cv = TimeSeriesSplit(
            gap=gap, max_train_size=None, n_splits=n_splits, test_size=None
        )
        scores = custom_cross_validate(
            BidirectionalGRU,
            SimpleImputer,
            StandardScaler,
            x,
            y,
            cv,
            timesteps,
            dates=dates,
            cement_types=df_copy[CEMENT_TYPES],
            estimator_params=None,
            imputer_params={"strategy": "median"},
        )
        scores = scores[0]
        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    scores = {key: np.array(val).flatten() for key, val in scores_final.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Time Series Split"
    results_dict_copy[
        "Cross Validation Params"
    ] = '{"N_Splits": 5, "Repeats": 3, "Gap": 0}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Blocking Time Series Split:
Repeats: 3
n_splits: 5



TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 1.435 (0.180)
MAE: 1.105 (0.128)
MAPE: 0.023 (0.003)
R2: 0.741 (0.104)


******
[TEST]
******
RMSE: 4.324 (3.271)
MAE: 3.172 (1.980)
MAPE: 0.068 (0.049)
R2: -1.590 (2.674)




TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.481 (0.286)
MAE: 1.201 (0.288)
MAPE: 0.025 (0.006)
R2: 0.661 (0.265)


******
[TEST]
******
RMSE: 4.258 (2.044)
MAE: 3.384 (1.585)
MAPE: 0.071 (0.038)
R2: -1.789 (3.051)




TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.352 (0.279)
MAE: 1.091 (0.259)
MAPE: 0.022 (0.005)
R2: 0.745 (0.176)


******
[TEST]
******
RMSE: 3.538 (1.243)
MAE: 2.738 (0.793)
MAPE: 0.056 (0.019)
R2: -1.195 (2.565)


Minutes Elapsed:  68.37587989171347


<IPython.core.display.Javascript object>

In [27]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features             Model            Cross Validation  \
                                                                              
0  Chemical + CS1 + CS3 + CS7  BidirectionalGRU  Blocking Time Series Split   
1  Chemical + CS1 + CS3 + CS7  BidirectionalGRU  Blocking Time Series Split   
2  Chemical + CS1 + CS3 + CS7  BidirectionalGRU  Blocking Time Series Split   
3  Chemical + CS1 + CS3 + CS7  BidirectionalGRU              Repeated KFold   
4  Chemical + CS1 + CS3 + CS7  BidirectionalGRU              Repeated KFold   
5  Chemical + CS1 + CS3 + CS7  BidirectionalGRU              Repeated KFold   
6  Chemical + CS1 + CS3 + CS7  BidirectionalGRU           Time Series Split   
7  Chemical + CS1 + CS3 + CS7  BidirectionalGRU           Time Series Split   
8  Chemical + CS1 + CS3 + CS7  BidirectionalGRU           Time Series Split   

  Timesteps RMSE Test            MAE Test           MAPE Test            \
                 mean       std      mean       std      mean       std   
0         1  5.060985  4.620092  3.845612  3.340128  0.081588  0.074701   
1         7  8.416255  8.409021  6.744304  6.625865  0.144470  0.147576   
2        14  7.765740  4.588193  6.509323  3.581415  0.136334  0.076461   
3         1  1.849068  0.114643  1.434239  0.077411  0.029153  0.001466   
4         7  1.955524  0.137611  1.531781  0.127570  0.031518  0.002694   
5        14  1.983682  0.171068  1.519980  0.134988  0.030562  0.002633   
6         1  4.323815  3.271390  3.171552  1.979623  0.067647  0.049433   
7         7  4.258287  2.043668  3.383794  1.584910  0.071191  0.037641   
8        14  3.538195  1.242715  2.737711  0.792724  0.056309  0.018638   

     R2 Test             
        mean        std  
0  -6.260210  13.822215  
1 -19.693881  34.956397  
2 -13.950874  17.362201  
3   0.790342   0.024612  
4   0.767533   0.034939  
5   0.744106   0.042596  
6  -1.589951   2.673628  
7  -1.788735   3.050623  
8  -1.194800   2.564998

<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 987
    2. Test: 247
<b>Total:</b> 1 model<br>

In [28]:
start = time.time()
test_size = 0.2
repeats = 3
TIMESTEPS_LIST = [1, 7, 14]

print("Out of time Cross Val:")
print(f"Repeats: {repeats}")
print(f"Train: {80}%", f"Test: {20}%")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    for _ in range(repeats):
        # Data Splitting
        x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
        y = df_copy["CS28"]

        x_train, x_test, y_train, y_test = train_test_split(
            x, y, test_size=test_size, random_state=SEED, shuffle=False
        )

        # Preprocessing
        imputer = SimpleImputer(strategy="median")
        scaler = StandardScaler()

        x_train = imputer.fit_transform(x_train)
        x_train = scaler.fit_transform(x_train)
        dates_train = dates[: x_train.shape[0]].reset_index(drop=True)
        cement_types_train = df_copy[CEMENT_TYPES][: x_train.shape[0]].reset_index(
            drop=True
        )

        x_test = imputer.transform(x_test)
        x_test = scaler.transform(x_test)
        dates_test = dates[x_train.shape[0] :].reset_index(drop=True)
        cement_types_test = df_copy[CEMENT_TYPES][x_train.shape[0] :].reset_index(
            drop=True
        )

        # Sequence Splitting
        data_train = pd.concat(
            [
                dates_train,
                pd.DataFrame(x_train, columns=x.columns),
                cement_types_train,
                y_train.reset_index(drop=True),
            ],
            axis=1,
        )
        data_test = pd.concat(
            [
                dates_test,
                pd.DataFrame(x_test, columns=x.columns),
                cement_types_test,
                y_test.reset_index(drop=True),
            ],
            axis=1,
        )

        x_train, y_train = split_sequences_per_cement_type(data_train, timesteps)
        x_test, y_test = split_sequences_per_cement_type(data_test, timesteps)

        # Train model and test evalutation
        # Fit model
        pipeline = Pipeline([("estimator", BidirectionalGRU())])
        pipeline.fit(x_train, y_train)

        # Make predictions
        y_train_pred = pipeline.predict(x_train)
        y_test_pred = pipeline.predict(x_test)

        # evaluate predictions
        scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    # scores = {key: val[0] for key, val in scores.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Out of time Split"
    results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(
        results_dict_copy, {key: value for key, value in scores_final.items()}
    )
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Out of time Cross Val:
Repeats: 3
Train: 80% Test: 20%



TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 1.532 (0.004)
MAE: 1.163 (0.003)
MAPE: 0.024 (0.000)
R2: 0.835 (0.001)


******
[TEST]
******
RMSE: 4.194 (0.083)
MAE: 3.952 (0.085)
MAPE: 0.073 (0.002)
R2: -3.237 (0.167)




TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.601 (0.122)
MAE: 1.308 (0.116)
MAPE: 0.027 (0.002)
R2: 0.821 (0.026)


******
[TEST]
******
RMSE: 3.726 (0.246)
MAE: 3.286 (0.240)
MAPE: 0.061 (0.004)
R2: -2.465 (0.449)




TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.192 (0.026)
MAE: 0.940 (0.018)
MAPE: 0.019 (0.000)
R2: 0.902 (0.004)


******
[TEST]
******
RMSE: 3.573 (0.213)
MAE: 3.211 (0.226)
MAPE: 0.059 (0.004)
R2: -2.241 (0.377)


Minutes Elapsed:  19.89423940976461


<IPython.core.display.Javascript object>

In [29]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)


Features             Model            Cross Validation  \
                                                                               
0   Chemical + CS1 + CS3 + CS7  BidirectionalGRU  Blocking Time Series Split   
1   Chemical + CS1 + CS3 + CS7  BidirectionalGRU  Blocking Time Series Split   
2   Chemical + CS1 + CS3 + CS7  BidirectionalGRU  Blocking Time Series Split   
3   Chemical + CS1 + CS3 + CS7  BidirectionalGRU           Out of time Split   
4   Chemical + CS1 + CS3 + CS7  BidirectionalGRU           Out of time Split   
5   Chemical + CS1 + CS3 + CS7  BidirectionalGRU           Out of time Split   
6   Chemical + CS1 + CS3 + CS7  BidirectionalGRU              Repeated KFold   
7   Chemical + CS1 + CS3 + CS7  BidirectionalGRU              Repeated KFold   
8   Chemical + CS1 + CS3 + CS7  BidirectionalGRU              Repeated KFold   
9   Chemical + CS1 + CS3 + CS7  BidirectionalGRU           Time Series Split   
10  Chemical + CS1 + CS3 + CS7  BidirectionalGRU           Time Series Split   
11  Chemical + CS1 + CS3 + CS7  BidirectionalGRU           Time Series Split   

   Timesteps RMSE Test            MAE Test           MAPE Test            \
                  mean       std      mean       std      mean       std   
0          1  5.060985  4.620092  3.845612  3.340128  0.081588  0.074701   
1          7  8.416255  8.409021  6.744304  6.625865  0.144470  0.147576   
2         14  7.765740  4.588193  6.509323  3.581415  0.136334  0.076461   
3          1  4.194086  0.082877  3.951909  0.084940  0.072831  0.001596   
4          7  3.726429  0.246217  3.286002  0.239739  0.060894  0.004457   
5         14  3.572581  0.212514  3.211008  0.226376  0.059338  0.004211   
6          1  1.849068  0.114643  1.434239  0.077411  0.029153  0.001466   
7          7  1.955524  0.137611  1.531781  0.127570  0.031518  0.002694   
8         14  1.983682  0.171068  1.519980  0.134988  0.030562  0.002633   
9          1  4.323815  3.271390  3.171552  1.979623  0.067647  0.049433   
10         7  4.258287  2.043668  3.383794  1.584910  0.071191  0.037641   
11        14  3.538195  1.242715  2.737711  0.792724  0.056309  0.018638   

      R2 Test             
         mean        std  
0   -6.260210  13.822215  
1  -19.693881  34.956397  
2  -13.950874  17.362201  
3   -3.237255   0.167072  
4   -2.464936   0.449043  
5   -2.241463   0.377111  
6    0.790342   0.024612  
7    0.767533   0.034939  
8    0.744106   0.042596  
9   -1.589951   2.673628  
10  -1.788735   3.050623  
11  -1.194800   2.564998

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [30]:
path = "../../../../../../reports/results/local_models/203/av/full/"
filename = f"BidirectionalGRU_results_full_{index_to_save}.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

In [31]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../../reports/results/local_models/203/av/grouped/"
filename = f"BidirectionalGRU_results_grouped_{index_to_save}.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>